In [23]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import OneHotEncoder

import pickle
from sklearn.decomposition import PCA

from sklearn.linear_model import LogisticRegression

from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB

from sklearn.neighbors import KNeighborsClassifier

from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import RandomForestClassifier

import xgboost as xgb

In [64]:
train = pd.read_csv("train_final.csv")
train_df = pd.DataFrame(train)

test = pd.read_csv("test_final.csv")
test_df = pd.DataFrame(test)

In [67]:
train_df.shape


(351815, 12)

In [68]:
test_df.shape

(89018, 12)

In [69]:
train_df.drop(columns='CustomerID', inplace=True)
train_df.columns = [col.lower().replace(' ', '_') for col in train_df.columns] 

In [70]:
test_df.drop(columns='CustomerID', inplace=True)
test_df.columns = [col.lower().replace(' ', '_') for col in test_df.columns] 

In [72]:
test_df

,age,gender,tenure,usage_frequency,support_calls,payment_delay,subscription_type,contract_length,total_spend,last_interaction,churn
0,27,Male,14,28,3,16,Standard,Monthly,862.00,9,1
1,40,Male,19,2,8,28,Standard,Monthly,620.81,21,1
2,27,Female,57,3,0,24,Standard,Annual,915.00,26,1
3,40,Male,21,14,0,11,Basic,Annual,592.83,9,0
4,39,Male,58,4,2,8,Standard,Monthly,694.00,15,1
...,...,...,...,...,...,...,...,...,...,...,...
89013,32,Male,29,28,3,18,Premium,Annual,613.39,9,0
89014,30,Female,20,10,2,20,Premium,Annual,836.80,7,0
89015,31,Male,42,11,5,23,Basic,Quarterly,502.00,10,1
89016,62,Male,15,4,0,24,Premium,Annual,345.00,8,1


In [73]:
test_df.dropna(inplace=True)

In [74]:
train_df.dropna(inplace=True)

In [75]:
train_df.shape

(351814, 11)

In [76]:
test_df.shape

(89018, 11)

In [78]:
def label_encoder(df):
    
    label_encoder = LabelEncoder()
    columns_to_encode=['gender','subscription_type','contract_length']
    for col in columns_to_encode:
        df[col] = label_encoder.fit_transform(df[col])
    descrete_col = ['age', 'tenure', 'usage_frequency', 'support_calls', 'payment_delay', 'last_interaction', 'churn']
    for col in descrete_col:
        df[col] = df[col].astype(int)    


In [79]:
label_encoder(train_df)
train_df

,age,gender,tenure,usage_frequency,support_calls,payment_delay,subscription_type,contract_length,total_spend,last_interaction,churn
0,44,1,16,16,4,28,1,1,109.00,29,1
1,63,1,2,8,6,2,0,1,204.00,4,1
2,37,1,15,16,10,19,2,1,113.00,17,1
3,60,0,6,19,9,11,2,1,764.38,18,1
4,19,1,6,9,8,5,0,2,494.00,13,1
...,...,...,...,...,...,...,...,...,...,...,...
351810,22,1,52,17,2,13,2,0,930.17,16,0
351811,45,1,12,11,2,13,2,0,976.76,12,0
351812,37,0,9,4,10,12,0,1,979.00,8,1
351813,33,1,27,5,10,20,1,0,824.00,1,1


In [80]:
label_encoder(test_df)
test_df

,age,gender,tenure,usage_frequency,support_calls,payment_delay,subscription_type,contract_length,total_spend,last_interaction,churn
0,27,1,14,28,3,16,2,1,862.00,9,1
1,40,1,19,2,8,28,2,1,620.81,21,1
2,27,0,57,3,0,24,2,0,915.00,26,1
3,40,1,21,14,0,11,0,0,592.83,9,0
4,39,1,58,4,2,8,2,1,694.00,15,1
...,...,...,...,...,...,...,...,...,...,...,...
89013,32,1,29,28,3,18,1,0,613.39,9,0
89014,30,0,20,10,2,20,1,0,836.80,7,0
89015,31,1,42,11,5,23,0,2,502.00,10,1
89016,62,1,15,4,0,24,1,0,345.00,8,1


In [85]:
X_train = train_df.drop(columns='churn')
Y_train = train_df['churn']
X_test = test_df.drop(columns='churn')
Y_test = test_df['churn']

X_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)
Y_train.reset_index(drop=True, inplace=True)
Y_test.reset_index(drop=True, inplace=True)

In [86]:
model = LogisticRegression(max_iter=1000 ,random_state=42)
model.fit(X_train ,Y_train)

LogisticRegression(max_iter=1000, random_state=42)

In [96]:
Y_pred_train = model.predict(X_train)
print(accuracy_score(Y_train ,Y_pred_train))

0.8508757468435025


In [97]:
Y_pred_test = model.predict(X_test)
print(accuracy_score(Y_test ,Y_pred_test))

0.852816284346986


In [98]:
cm_train = confusion_matrix(Y_train, Y_pred_train)

print("Confusion Matrix for training:")
print(cm_train)

Confusion Matrix for training:
[[129063  23263]
 [ 29201 170287]]


In [99]:
cm_test = confusion_matrix(Y_test, Y_pred_test)

print("Confusion Matrix for test:")
print(cm_test)

Confusion Matrix for test:
[[32763  5744]
 [ 7358 43153]]
